In [114]:
import nltk
from nltk.stem import WordNetLemmatizer
import argparse
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.feature_extraction import DictVectorizer
import pandas as pd
from sklearn.metrics import classification_report
import argparse


Function for the extraction of differenct features

In [93]:
#inspire from from different documentations and stack over flow questions

def wordnet_pos(pos):

    if pos.startswith('J'):
        return 'a'
    elif pos.startswith('V'):
        return 'v'
    elif pos.startswith('N'):
        return 'n'
    elif pos.startswith('R'):
        return 'r'
    else:
        return 'n'

def lemma_extraction(tokens, pos_list):
    lemmas = []
    lem = WordNetLemmatizer()
    for token, pos in zip(tokens, pos_list):
        lemma = lem.lemmatize(token, wordnet_pos(pos))
        lemmas.append(lemma)


    return lemmas

def pos_extraction(tokens):
    pos_list = []
    tagged = nltk.pos_tag(tokens)
    for token, pos_tag in tagged:
        pos_list.append(pos_tag)   

    return pos_list

def previous_and_next_token_extraction(tokens):
    position_index = 0

    prev_tokens = []
    next_tokens = []

    for i in range(len(tokens)):

        prev_index = (position_index - 1)
        next_index = (position_index + 1)

        #previous token
        if prev_index < 0:
            previous_token = "None"
        else: 
            previous_token = tokens[prev_index]

        prev_tokens.append(previous_token)

        #next token
        if next_index < len(tokens):
            next_token = tokens[next_index]
        else: 
            next_token = "None"

        next_tokens.append(next_token)

        #moving to next token in list 
        position_index += 1

    return prev_tokens, next_tokens



def neg(tokens, neg):
    neg_word_list = []

    for token in tokens:

        # label 1 if token is in negative word list
        if token in neg:
            label = 1

        # label 0 if token is not in negative word list
        else:
            label = 0
        
        neg_word_list.append(label)

    return neg_word_list

Extracting features with the help of above functions

In [94]:

def file(input_file):
    # open and red file and returns token, gold labels and chapters 
    
    df = pd.read_csv(input_file, encoding='utf-8', delimiter = "\t", quotechar = '|')
    chapters = df.iloc[:, 0]
    sent_id = df.iloc[:, 1]
    token_position = df.iloc[:, 2]
    tokens = df.iloc[:, 3]
    gold = df.iloc[:, 4]
    
    return tokens, gold, chapters, 

def write_features(tokens,gold):
    
    negl= ['nor', 'neither', 'without', 'nobody', 'none', 'nothing', 'never', 'not', 'no', 'nowhere', 'non']
    #not considering at this moment, will work on the smooth implimentation of this
    # written the code for negetion words, fucntion can be seen above
    # had few difificulties in final processing of output 
    
    
    # Defining header names
    feature_names = ["token",
                "lemma",
                "pos_tag",
                "prev_token",
                "next_token",
                    "neg"]

   # using all above mentioned functions for the extraction of features, except fucntion neg  
    pos_tag = pos_extraction(tokens)
    lemma = lemma_extraction(tokens, pos_tag)
    neg_word = neg(tokens,negl)
    prev_next_token = previous_and_next_token_extraction(tokens)
    prev_tokens, next_token = prev_next_token

    features_dict = {'token': tokens, 'pos_tag': pos_tag,'lemma': lemma,'neg_word':neg_word ,'prev_token': prev_tokens,
                     'next_token': next_token}
    return features_dict
        

Creatinf featues list, svm model and prdictions 

In [95]:

def data(input_file):
    
    data = []
    tokens, gold, chapters= file(input_file) # from def file mentioned above
    feature_dict = write_features(tokens,gold) # from above function
    #features 
    token = feature_dict['token']
    lemma = feature_dict['lemma']
    pos_tag = feature_dict['pos_tag']
    neg_word =  feature_dict['neg_word'],
    prev_token= feature_dict['prev_token']
    next_token= feature_dict['next_token']
    golds = gold.tolist() #gold labels to list
    token = token.tolist() #tokens to list 
    
    #print(type(next_token)
    #,type(token),type(lemma),type(pos_tag),type(neg_word),
    # type(prev_token),type(next_token))
    #print(type(list(neg_word)))
    for token,lemma,pos_tag,prev_token,next_token in zip(token,lemma,pos_tag,
                                                                         prev_token,next_token):
        feature_dict = {"Tokens": token, 'lemmas':lemma,'pos_tags':pos_tag,
                        'prev_token':prev_token,'next_token':next_token}

        #feature_dict = {"1": token, '2':lemma,'3':pos_tag,'4':neg_word,'5':prev_token,'6':next_token, '7': gold}
        data.append(feature_dict)
        #print(data)
    return data, golds

def create_classifier(train_features, train_targets: list):
    #creating SVM model 
    
    model = LinearSVC()
    vec = DictVectorizer()
    featuresv = vec.fit_transform(training_features)
    train_targets = np.array(train_targets)
    model.fit(featuresv, train_targets)

    return model, vec



def predicted(testfile, model,vec):
    features, goldlabels = data(testfile)
    test_features_vectorized = vec.transform(features)
    predictions = model.predict(test_features_vectorized)

    return predictions, goldlabels


Importing Datasets

In [ ]:
training_file = 'data/no_dev.txt'
dev_file = 'data/dev.txt'
test_file_1 = 'data/test_1.txt' #cardboard test
test_file_2 = 'data/test_2.txt' #circle test

In [96]:
#generating TRAINING FEATURES 
training_features, gold_labels= data(training_file)


<class 'list'> <class 'list'> <class 'list'> <class 'list'> <class 'tuple'> <class 'list'> <class 'list'>
<class 'list'>


In [84]:
 #creating model and vectorzor 
model, vec= create_classifier(training_features, gold_labels)

In [106]:
# develoment predictions 
dev, gold = predicted(dev_file, model, vec)
# test predictions 1
test1, gold1 = predicted(test_file_1, model, vec)
# test predictions 2
test2, gold2 = predicted(test_file_2, model, vec)

<class 'list'> <class 'list'> <class 'list'> <class 'list'> <class 'tuple'> <class 'list'> <class 'list'>
<class 'list'>


Classification report and P,R,f-1scores

In [113]:
def print_confusion_matrix(predictions, goldlabels):
    # based on example from https://datatofish.com/confusion-matrix-python/
    data = {'Gold': goldlabels, 'Predicted': predictions}
    df = pd.DataFrame(data, columns=['Gold', 'Predicted'])

    confusion_matrix = pd.crosstab(df['Gold'], df['Predicted'], rownames=['Gold'], colnames=['Predicted'])
    print(confusion_matrix)
    report = classification_report(goldlabels,predictions,digits = 3)
    print('METRICS: ')
    print()
    print(report)

Development se classification report

In [108]:
cf_dev = print_confusion_matrix(dev,gold)

Predicted  B-NEG  I-NEG      O
Gold                          
B-NEG        149      0     27
I-NEG          0      2      1
O              6      0  13381
METRICS: 

              precision    recall  f1-score   support

       B-NEG      0.961     0.847     0.900       176
       I-NEG      1.000     0.667     0.800         3
           O      0.998     1.000     0.999     13387

    accuracy                          0.997     13566
   macro avg      0.986     0.838     0.900     13566
weighted avg      0.997     0.997     0.997     13566



Test set one, classification report

In [111]:
cf_test_1 = print_confusion_matrix(test1,gold1)

Predicted  B-NEG      O
Gold                   
B-NEG        120     14
I-NEG          0      1
O              6  10042
METRICS: 

              precision    recall  f1-score   support

       B-NEG      0.952     0.896     0.923       134
       I-NEG      0.000     0.000     0.000         1
           O      0.999     0.999     0.999     10048

    accuracy                          0.998     10183
   macro avg      0.650     0.632     0.641     10183
weighted avg      0.998     0.998     0.998     10183



/Users/rishvikchandel/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rishvikchandel/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rishvikchandel/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

Test set two, classification report

In [112]:
cf_test_2 = print_confusion_matrix(test2,gold2)

Predicted  B-NEG     O
Gold                  
B-NEG        127     8
I-NEG          0     4
O              7  8885
METRICS: 

              precision    recall  f1-score   support

       B-NEG      0.948     0.941     0.944       135
       I-NEG      0.000     0.000     0.000         4
           O      0.999     0.999     0.999      8892

    accuracy                          0.998      9031
   macro avg      0.649     0.647     0.648      9031
weighted avg      0.997     0.998     0.998      9031



/Users/rishvikchandel/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rishvikchandel/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/rishvikchandel/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r